In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_key = os.getenv("TAVILY_API_KEY")

In [6]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

class State(TypedDict):
    messages: Annotated[list, add_messages]

tool = TavilySearchResults(max_result=2)
tools = [tool]
tool_node = ToolNode(tools)

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    result = llm_with_tools.invoke(state["messages"])
    return {"messages" : [result]}


graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile(checkpointer=memory)


In [8]:
config = { "configurable": {"thread_id": "1"}}
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "q", "exit"]:
        print("Good bye")
        break
    for event in graph.stream({"messages": ("user", user_input)}, config):
        for value in event.values():
            print("Assistant: ", value["messages"][-1].content)

Assistant:  
Assistant:  [{"url": "https://www.taskade.com/blog/top-autonomous-agents/", "content": "🐑🤖\nCategorized in:\nShare Article:\nRelated Articles\nTable of ContentsToggle Table of ContentToggle\nLike what you read?\nSubscribe to our email newsletter to get the latest posts delivered right to your email. Visit the repo page to learn more: https://github.com/microsoft/autogen\nMetaGPT\nMetaGPT is another framework for open-source AI agents that attempts to imitate the structure of a traditional software company. A (relatively ) hands-free AI experience.\n💡 AI Agent Trivia: The concept of autonomous AI agents came to life with a paper titled “Task-Driven Autonomous Agent” published in early 2023 by Yohei Nakajima, general partner at Untapped Capital.\n Visit the repo page to learn more: https://github.com/microsoft/JARVIS\nOpenAGI\nOpenAGI is an open-source AGI (artificial general intelligence) research platform combining small, expert models — models tailored for tasks like sent

In [9]:
snapshot = graph.get_state(config)
snapshot

StateSnapshot(values={'messages': [HumanMessage(content='There are serveral AI-agent development frameworks, for example langgrah. Show the ranking of these frameworks by usages rate.', additional_kwargs={}, response_metadata={}, id='f0305a65-a80c-4971-ab10-ecdf01fe7ce0'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uBGVEPj8skbmgMlBZA5kx3ll', 'function': {'arguments': '{"query":"AI agent development frameworks rankings by usage rate 2023"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 104, 'total_tokens': 133, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c019

In [11]:
graph.invoke({"messages": [{"role": "user", "content": "내가 한 첫 질문이 뭐였어"}]}, config)

{'messages': [HumanMessage(content='There are serveral AI-agent development frameworks, for example langgrah. Show the ranking of these frameworks by usages rate.', additional_kwargs={}, response_metadata={}, id='f0305a65-a80c-4971-ab10-ecdf01fe7ce0'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uBGVEPj8skbmgMlBZA5kx3ll', 'function': {'arguments': '{"query":"AI agent development frameworks rankings by usage rate 2023"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 104, 'total_tokens': 133, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c01906d2-7ff3-41b4-a84d

In [14]:
config2 = {"configurable": {"thread_id": "2"}}
graph.invoke({"messages": [{"role": "user", "content": "내가 한 첫 질문이 뭐였어"}]}, config2)

{'messages': [HumanMessage(content='내가 한 첫 질문이 뭐였어', additional_kwargs={}, response_metadata={}, id='7696e6d4-6364-42b3-938c-63d70207328e'),
  AIMessage(content='죄송하지만, 이전 대화 내용이나 질문을 기억할 수는 없습니다. 하지만 새로 질문해 주시면 최선을 다해 도와드리겠습니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 88, 'total_tokens': 124, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-6e90aa97-bf18-41b3-b069-72755d8445f7-0', usage_metadata={'input_tokens': 88, 'output_tokens': 36, 'total_tokens': 124, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

기억할 메시지 개수 제한하기

In [15]:
def filter_messages(messages: list):
    return messages[-2:]

def chatbot(state: State):
    messages = filter_messages(state["messages"])
    result = llm_with_tools.invoke(messages)
    return {"messages" : [result]}

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile(checkpointer=memory)

In [16]:
from langchain_core.messages import HumanMessage

config = { "configurable": {"thread_id": "20"}}
input_message = HumanMessage(content="hi! I'm bob and I like soccer")

for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

input_message = HumanMessage(content="what's my name")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()
input_message = HumanMessage(content="what's my name")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()
input_message = HumanMessage(content="what's my favorite?")
for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print() 


================================ Human Message =================================

hi! I'm bob and I like soccer
================================== Ai Message ==================================

Hi Bob! It's great to meet a fellow soccer fan! Do you have a favorite team or player?
================================ Human Message =================================

what's my name
================================== Ai Message ==================================

You mentioned your name is Bob. How can I assist you today?
================================ Human Message =================================

what's my name
================================== Ai Message ==================================

Your name is Bob. How can I assist you today?
================================ Human Message =================================

what's my favorite?
================================== Ai Message ==================================

I don't have information about your preferences or favorites. However, 

In [17]:
# user interrupt before run tools
graph = graph_builder.compile(checkpointer=memory, interrupt_before=["tools"])

In [18]:
config = { "configurable": {"thread_id": "30"}}
input_message = HumanMessage(content="what's the langgraph")

for event in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()


================================ Human Message =================================

what's the langgraph
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_odJDfyhG8rQM6OAwDwOm5x0e)
 Call ID: call_odJDfyhG8rQM6OAwDwOm5x0e
  Args:
    query: langgraph
